In [2]:
#Chargement et Préparation des Données

import numpy as np
import h5py
from sklearn.model_selection import train_test_split

# Charger les données
with h5py.File(r'C:\Users\CyberVortex\PFE_Decoding\dataset\ldpc_dataset.h5', 'r') as hf:
    X = hf['code_word'][:]  # (40000, 200)
    y = hf['message'][:]     # (40000, 100)

# Normalisation globale
X_mean, X_std = np.mean(X), np.std(X)
X = (X - X_mean) / X_std

# Séparation entraînement/validation (80/20)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Redimensionner pour les RNN (ajouter une dimension de canal)
X_train = X_train[..., np.newaxis]  # (32000, 200, 1)
X_val = X_val[..., np.newaxis]       # (8000, 200, 1)

In [3]:
# Architecture du Modèle 

from tensorflow.keras.layers import Input, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

def build_model(input_shape=(200, 1), k=100):
    inputs = Input(shape=input_shape)
    x = Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01)))(inputs)
    x = Dropout(0.3)(x)
    x = Bidirectional(LSTM(64, kernel_regularizer=l2(0.01)))(x)
    x = Dropout(0.3)(x)
    outputs = Dense(k, activation='sigmoid')(x)
    return Model(inputs, outputs)

model = build_model()

In [6]:
#Métriques Personnalisées (BER, FER)

import tensorflow as tf

def ber(y_true, y_pred):
    """Bit Error Rate (BER)"""
    y_true = tf.cast(y_true, tf.float32)  # Convertir y_true en float32
    errors = tf.cast(y_true != tf.round(y_pred), tf.float32)
    return tf.reduce_mean(errors)

def fer(y_true, y_pred):
    """Frame Error Rate (FER)"""
    y_true = tf.cast(y_true, tf.float32)  # Convertir y_true en float32
    frame_errors = tf.cast(tf.reduce_any(y_true != tf.round(y_pred), axis=1), tf.float32)
    return tf.reduce_mean(frame_errors)

In [ ]:
#Compilation et Entraînement avec Callbacks

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


# Compiler le modèle
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[ber, fer, 'accuracy']
)

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Callbacks
callbacks = [
    EarlyStopping(
        patience=5, 
        restore_best_weights=True, 
        monitor='val_ber', 
        mode='min'  # <-- Ajoutez ceci
    ),
    ReduceLROnPlateau(
        factor=0.2, 
        patience=3, 
        monitor='val_loss', 
        mode='min'  # <-- Ajoutez ceci
    ),
    ModelCheckpoint(
        'best_model.h5', 
        save_best_only=True, 
        monitor='val_ber', 
        mode='min'  # <-- Ajoutez ceci
    )
]

# Entraînement
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=128,
    validation_data=(X_val, y_val),
    callbacks=callbacks
)

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0393 - ber: 0.4573 - fer: 1.0000 - loss: 0.6869

250/250 ━━━━━━━━━━━━━━━━━━━━ 303s 1s/step - accuracy: 0.0393 - ber: 0.4573 - fer: 1.0000 - loss: 0.6869 - val_accuracy: 0.0816 - val_ber: 0.4406 - val_fer: 1.0000 - val_loss: 0.6827 - learning_rate: 0.0010
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0516 - ber: 0.4419 - fer: 1.0000 - loss: 0.6831

250/250 ━━━━━━━━━━━━━━━━━━━━ 264s 1s/step - accuracy: 0.0516 - ber: 0.4419 - fer: 1.0000 - loss: 0.6831 - val_accuracy: 0.0531 - val_ber: 0.4341 - val_fer: 1.0000 - val_loss: 0.6806 - learning_rate: 0.0010
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0530 - ber: 0.4352 - fer: 1.0000 - loss: 0.6811

250/250 ━━━━━━━━━━━━━━━━━━━━ 271s 1s/step - accuracy: 0.0530 - ber: 0.4352 - fer: 1.0000 - loss: 0.6811 - val_accuracy: 0.0536 - val_ber: 0.4312 - val_fer: 1.0000 - val_loss: 0.6797 - learning_rate: 0.0010
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0550 - ber: 0.4319 - fer: 1.0000 - loss: 0.6800

250/250 ━━━━━━━━━━━━━━━━━━━━ 276s 1s/step - accuracy: 0.0551 - ber: 0.4319 - fer: 1.0000 - loss: 0.6800 - val_accuracy: 0.0766 - val_ber: 0.4278 - val_fer: 1.0000 - val_loss: 0.6783 - learning_rate: 0.0010
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0604 - ber: 0.4306 - fer: 1.0000 - loss: 0.6794

250/250 ━━━━━━━━━━━━━━━━━━━━ 287s 1s/step - accuracy: 0.0604 - ber: 0.4306 - fer: 1.0000 - loss: 0.6794 - val_accuracy: 0.0558 - val_ber: 0.4268 - val_fer: 1.0000 - val_loss: 0.6781 - learning_rate: 0.0010
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0683 - ber: 0.4285 - fer: 1.0000 - loss: 0.6788

250/250 ━━━━━━━━━━━━━━━━━━━━ 291s 1s/step - accuracy: 0.0683 - ber: 0.4285 - fer: 1.0000 - loss: 0.6788 - val_accuracy: 0.0650 - val_ber: 0.4255 - val_fer: 1.0000 - val_loss: 0.6776 - learning_rate: 0.0010
Epoch 7/50
111/250 ━━━━━━━━━━━━━━━━━━━━ 2:44 1s/step - accuracy: 0.0618 - ber: 0.4257 - fer: 1.0000 - loss: 0.6778